In [1]:
import os
import pandas as pd
import glob
import h5py
import numpy as np
from keras.preprocessing.image import *
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import optimizers
# from keras import backend as K

Using TensorFlow backend.


In [49]:
# path to the model weights file.
weights_path = './vgg16_weights.h5'
top_model_weights_path = './fon_top.h5'

top_dir = '/data/nlp/corpora/twitter4food/'
data_dir = '/data/nlp/corpora/twitter4food/foodOrNot'
twitter_dir = '/data/nlp/corpora/twitter4food/twitterImages'
insta_dir = '/data/nlp/corpora/twitter4food/externalImages'

img_width = 224
img_height = 224

In [3]:
# https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def randomOrder(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

def load_data(path='/data/nlp/corpora/twitter4food/foodOrNot', use_test=False):
    extensions = ['*.JPEG', '*.jpg', '*.png', '*.gif']
    
    fs = []
    for ext in extensions:
        fs.extend(glob.glob(path + '/train/food/' + ext))
    tr_pos = [img_to_array(load_img(fn, target_size=(img_height, img_width))) for fn in fs]
    fs = []
    for ext in extensions:
        fs.extend(glob.glob(path + '/train/notFood/' + ext))
    tr_neg = [img_to_array(load_img(fn, target_size=(img_height, img_width))) for fn in fs]
    tr_labels = np.array((len(tr_pos) * [1]) + (len(tr_neg) * [0]))
    tr = np.array(tr_pos + tr_neg)
    print('training shape: ', tr.shape)
    tr = preprocess_input(tr)
    (tr, tr_labels) = randomOrder(tr, tr_labels)

    if (not use_test):
        folder = 'dev'
    else:
        folder = 'test'

    ffs = []
    nffs = []
    for ext in extensions:
        ffs.extend(glob.glob(path + '/' + folder + '/food/' + ext))
        nffs.extend(glob.glob(path + '/' + folder + '/notFood/' + ext))

    val_pos = [img_to_array(load_img(fn, target_size=(img_height, img_width))) for fn in ffs]
    val_neg = [img_to_array(load_img(fn, target_size=(img_height, img_width))) for fn in nffs]

    val_labels = np.array((len(val_pos) * [1]) + (len(val_neg) * [0]))
    val = np.array(val_pos + val_neg)
    print('validation shape: ', val.shape)
    val = preprocess_input(val)
    (val, val_labels) = randomOrder(val, val_labels)

    return (tr, tr_labels), (val, val_labels)


In [4]:
nb_classes = 2

(X_train, y_train), (X_val, y_val) = load_data(data_dir, use_test=False)

## This is taken care of by img_to_array
# if K.image_dim_ordering() == 'tf':
#     X_train = np.rollaxis(X_train, 1, 3)
#     X_dev = np.rollaxis(X_dev, 1, 3)
#     X_test = np.rollaxis(X_test, 1,3)
#     input_shape = (None, None, channels)
# else:
#     input_shape = (channels, None, None)

# y_train = np.array(y_train)
# y_val = np.array(y_val)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'validation samples')

# convert class vectors to n-ary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_val = np_utils.to_categorical(y_val, nb_classes)

print('y_train shape:', y_train.shape)

training shape:  (55305, 224, 224, 3)
validation shape:  (6914, 224, 224, 3)
X_train shape: (55305, 224, 224, 3)
55305 train samples
6914 validation samples
y_train shape: (55305, 2)


In [6]:
batch_size = 32

In [14]:
nb_train_samples = y_train.shape[0]
nb_validation_samples = y_val.shape[0]

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator()

model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [8]:
# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
import h5py
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

In [9]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu', name='dense1'))
top_model.add(Dropout(0.5, name='dropout'))
top_model.add(Dense(2, activation='softmax', name='dense2'))

#top_model.load_weights(top_model_weights_path)

model_vgg.add(top_model)

In [10]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [11]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [12]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_val, y_val, batch_size=batch_size)

In [15]:
nb_epoch = 2

# fine-tune the model
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/2
55305/55305 [==============================] - 10577s - loss: 0.2361 - acc: 0.9662 - val_loss: 0.0549 - val_acc: 0.9858

In [16]:
model_vgg.save_weights('fon_vgg.h5')

In [ ]:
model_vgg.load_weights('fon_vgg.h5')

In [17]:
model_vgg.evaluate_generator(validation_generator, nb_validation_samples)

[0.072081715430217116, 0.98264391090540937]

In [21]:
# just clearing up memory
del X_train
del y_train
del X_val
del y_val

NameError: name 'reset_selective' is not defined

In [39]:
from ipywidgets import FloatProgress
from IPython.display import display

In [44]:
# expects a folder for each user with multiple images per folder
def annotate(path):
    annotations = {}
    
    ids = [name for name in os.listdir(path)
           if os.path.isdir(os.path.join(path, name))]
    
    extensions = ['*.JPG', '*.JPEG', '*.jpg', '*.jpeg']

    prog = FloatProgress(min=0, max=len(ids))
    display(prog)

    for user in ids:
        fs = []
        for ext in extensions:
            fs.extend(glob.glob(os.path.join(path, user, ext)))
        userimgs = [img_to_array(load_img(fn, target_size=(img_height, img_width))) for fn in fs]
        if(len(userimgs) > 0):
            userimgs = preprocess_input(np.array(userimgs))
            preds = model_vgg.predict_classes(userimgs, verbose=0)
            annotations[user] = preds.mean()
        prog.value += 1

    prog.close()

    return annotations


In [ ]:
twitter_annos = annotate(twitter_dir)

In [50]:
twout = open(os.path.join(top_dir, 'twitterFoodPerc.tsv'), 'w')
for u, p in twitter_annos.items():
    twout.write(u + '\t' + "%.2f" % p + '\n')
twout.close()

In [ ]:
insta_annos = annotate(insta_dir)

In [52]:
igout = open(os.path.join(top_dir, 'instaFoodPerc.tsv'), 'w')
for u, p in insta_annos.items():
    igout.write(u + '\t' + "%.2f" % p + '\n')
igout.close()